In [4]:
### calculate daily ERA5 wind speed from u- and v-components and place onto EASE 2.0 grid

In [5]:
import numpy as np
import tqdm
from netCDF4 import Dataset
import itertools
from math import sqrt
from regrid import regrid
import xarray as xr
import warnings
warnings.filterwarnings("ignore")

In [6]:
# creating numpy are with daily wind data on EASE grid for 2010-2020

datapath = '/Users/carmennab/Dropbox/OI_PolarSnow/data/ERA5/wind/'
EASE_lats = np.load('/Users/carmennab/Dropbox/OI_PolarSnow/EASE/auxiliary/new_lat_25km.npy')
EASE_lons = np.load('/Users/carmennab/Dropbox/OI_PolarSnow/EASE/auxiliary/new_lon_25km.npy')
missing = np.full((153,360,360),np.nan)
years = ['2010', '2011', '2012','2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

all_speed = {}

for year in tqdm.tqdm(years):
    
    # load ERA5 temperature data
    file = datapath + year +'.nc'
    data = Dataset(file,'r')
    lon = data['longitude']
    lat = data['latitude']
    era5_lon,era5_lat = np.meshgrid(np.array(lon),np.array(lat))
    wind_v = data['v10']
    wind_u = data['u10'] 
          
    # turn hourly data into daily data and regrid onto EASE grid        
    v_daily = []
    u_daily = []
    speed_daily = []

    for i in np.arange(0,wind_v.shape[0],24):

        # v component

        v_on_day = wind_v[i:i+24]

        v_day_ave = np.nanmean(v_on_day,axis=0)
        
        # u component

        u_on_day = wind_v[i:i+24]

        u_day_ave = np.nanmean(u_on_day,axis=0)

        speed = np.full((121,1440),np.nan)
        for i, j in itertools.product(np.arange(121),
                              np.arange(1440)):
            speed[i,j] = sqrt((v_day_ave[i,j]**2) + (u_day_ave[i,j]**2))
        speed_WG = regrid(speed, era5_lon, era5_lat, EASE_lons, EASE_lats, method = 'linear')
        speed_daily.append(speed_WG)
    
    speeds = np.concatenate((speed_daily[:-92],missing,speed_daily[-92:]),axis=0)
    all_speed[year] = speeds

100%|████████████████████████████████████████| 11/11 [1:52:10<00:00, 611.86s/it]


In [7]:
array_speed = np.concatenate((all_speed['2010'],all_speed['2011'],all_speed['2012'],all_speed['2013'],all_speed['2014'],
               all_speed['2015'],all_speed['2016'],all_speed['2017'],all_speed['2018'],all_speed['2019'],
               all_speed['2020']),axis=0)

In [8]:
# create wind NETCDF
days = np.arange(0,4018)
data_vars = {'Wind Speed':(['t','x','y'],array_speed)}
ds = xr.Dataset(data_vars = data_vars,
                coords={'Longitude':(['x','y'],EASE_lons),
                        'Latitude':(['x','y'],EASE_lats),
                         'Day':(['t'],days)}     )
ds.to_netcdf(f'/Users/carmennab/Dropbox/OI_PolarSnow/EASE/data/ERA5_windspeed.nc','w')